In [1]:
import pymongo
CONNECTION_STRING = "mongodb://cstring"
client = pymongo.MongoClient(CONNECTION_STRING)
DATABASE_NAME_1 = "aug2023"
mydb = client[DATABASE_NAME_1]
source_collection = mydb.track_ride

In [2]:
ids=[218192,249835,276383,314398,343456,358524,365892,378004,395542,399166,404465,404501,416351,421562,423332,425009,449973,475882,483370,494281,498078,499231,501399,503207,503573,504518,511896,516998,519846,521367,524864,525189,525482,525946,528113,529638,529651,530031,530322,530475,530476,530514,530634,530710,531024,531266,531308,531355,531546,531577,531591,531615,531790,531918,532018,532036,532066,532095,532944,533346
]

In [3]:
import pandas as pd
data = source_collection.find({'userId': {'$in': ids}}, {'userId', 'latitude', 'longitude', 'createdTime','lockStatus'})
d = pd.DataFrame(list(data))

In [5]:
d.nunique()

_id            103820
userId             60
latitude        48626
longitude       39850
lockStatus          2
createdTime    103801
dtype: int64

In [6]:
d

,_id,userId,latitude,longitude,lockStatus,createdTime
0,64cb8446e4b02e2723d3d5d5,218192,28.485941,77.058525,0,2023-08-03 10:41:10.184
1,64cb8691e4b02e2723d60f5f,218192,28.485868,77.057937,1,2023-08-03 10:50:57.473
2,64cb9d2fe4b02e2723ecd915,218192,28.485893,77.057968,0,2023-08-03 12:27:27.880
3,64cba225e4b02e2723f23063,218192,28.486091,77.058090,0,2023-08-03 12:48:37.365
4,64cba2ebe4b02e2723f30bf3,218192,28.486425,77.058105,0,2023-08-03 12:51:55.724
...,...,...,...,...,...,...
103815,64ce09a2e4b02e272594888b,533346,28.617395,77.385216,1,2023-08-05 08:34:42.650
103816,64ce0a09e4b02e272594fe6b,533346,28.617223,77.385238,0,2023-08-05 08:36:25.184
103817,64ce0aefe4b02e2725960829,533346,28.615284,77.385101,0,2023-08-05 08:40:15.053
103818,64ce0b32e4b02e27259656d5,533346,28.613626,77.386162,0,2023-08-05 08:41:22.235


In [10]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [11]:
d['createdTime'].dt.hour

0         10
1         10
2         12
3         12
4         12
          ..
103815     8
103816     8
103817     8
103818     8
103819     8
Name: createdTime, Length: 103820, dtype: int64

In [12]:
time_intervals = [(0, 6), (6, 12), (12, 18), (18, 24)]  # In hours

result_df = pd.DataFrame(columns=['userId', 'work_latitude', 'work_longitude', 'home_latitude', 'home_longitude'])

In [13]:
def analyze_driver_data(driver_data):
    driver_id = driver_data['userId'].iloc[0]
    driver_results = {'userId': driver_id}
    
    for start_hour, end_hour in time_intervals:
        
        time_interval_data = driver_data[(driver_data['createdTime'].dt.hour >= start_hour) &
                                         (driver_data['createdTime'].dt.hour < end_hour)]
        
        
        if len(time_interval_data) >= 2:  
        
            kmeans = KMeans(n_clusters=2)
            time_interval_data['cluster'] = kmeans.fit_predict(time_interval_data[['latitude', 'longitude']])

            
            home_cluster = time_interval_data[time_interval_data['cluster'] == 0]
            work_cluster = time_interval_data[time_interval_data['cluster'] == 1]

           
            if not home_cluster.empty:
                home_location = home_cluster[['latitude', 'longitude']].mean()
                driver_results['home_latitude'] = home_location['latitude']
                driver_results['home_longitude'] = home_location['longitude']

            if not work_cluster.empty:
                work_location = work_cluster[['latitude', 'longitude']].mean()
                driver_results['work_latitude'] = work_location['latitude']
                driver_results['work_longitude'] = work_location['longitude']
    
    
    result_df.loc[len(result_df)] = driver_results


grouped_data = d.groupby('userId')
for driver_id, driver_data in grouped_data:
    analyze_driver_data(driver_data)


C:\Users\gcv\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\gcv\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
C:\Users\gcv\AppData\Local\Temp\ipykernel_7120\1839602128.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_interval_data['cluster'] = kmeans.fit_predict(time_interval_data[['latitude', 'lo

In [14]:
result_df

,userId,work_latitude,work_longitude,home_latitude,home_longitude
0,218192,28.418879,77.064922,28.473407,77.079655
1,249835,28.395505,77.052568,28.412201,77.030916
2,276383,28.404044,77.039965,28.446292,77.061905
3,314398,28.465204,77.077121,28.422895,77.153476
4,343456,28.395173,77.040607,28.392495,77.041845
5,358524,28.661854,77.391049,28.677004,77.469399
6,365892,28.394487,77.284942,28.367353,77.287619
7,378004,28.424016,77.150821,28.465437,77.077037
8,395542,28.544785,77.395925,28.515957,77.389892
9,399166,28.483649,77.068097,28.483882,77.091570


In [15]:
result_df.nunique()

userId            60
work_latitude     60
work_longitude    60
home_latitude     60
home_longitude    60
dtype: int64

In [ ]:
map_center = [df['latitude'].mean(), df['longitude'].mean()]
mymap = folium.Map(location=map_center, zoom_start=2)

# # Define colors for each user_id
# colors = {7406: 'red', 25761: 'blue', 11762: 'green'}  # Add more user IDs and colors as needed

In [33]:
import folium

# Function to filter and plot the user's journey
def plot_user_journey(user_id, journey_color):
    user_data = d[d['userId'] == user_id]
    
    for index, row in user_data.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            color=journey_color,  # Use the same color for the journey as the marker
            fill=True,
            fill_color=journey_color,  # Use the same color for the fill
            fill_opacity=0.7,
            popup=f"User ID: {row['userId']}").add_to(mymap)
    
    if user_id in result_df['userId'].values:
        user_info = result_df[result_df['userId'] == user_id].iloc[0]
        work_popup = folium.Popup(f"Rider ID: {user_id}<br>Location: Work", parse_html=True)
        folium.Marker([user_info['work_latitude'], user_info['work_longitude']], icon=folium.Icon(color=journey_color), popup=work_popup).add_to(mymap)
    
        home_popup = folium.Popup(f"Rider ID: {user_id}<br>Location: Home", parse_html=True)
        folium.Marker([user_info['home_latitude'], user_info['home_longitude']], icon=folium.Icon(color=journey_color), popup=home_popup).add_to(mymap)

# Create a map centered at the average latitude and longitude
map_center = [d['latitude'].mean(), d['longitude'].mean()]
mymap = folium.Map(location=map_center, zoom_start=2)

import random

# List of user IDs to plot
user_ids_to_plot = random.sample(ids, 5)

color_names = ['red', 'blue', 'green', 'orange', 'purple', 'pink', 'yellow', 'cyan']

# Randomly select 5 user IDs from the list
user_ids_to_plot = random.sample(ids, 5)

# Loop through the user IDs and plot journeys and locations with the same color names
for user_id_to_plot, journey_color in zip(user_ids_to_plot, color_names):
    plot_user_journey(user_id_to_plot, journey_color)

# Save the map to an HTML file
mymap.save('test.html')
